In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.metrics import mean_squared_error

#R values
df_r_all = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
df_r_all.date = pd.to_datetime(df_r_all.date)

In [ ]:
#google
df2020 = pd.read_csv("Region_Mobility_Report_CSVs/2020_DE_Region_Mobility_Report.csv")
df2021 = pd.read_csv("Region_Mobility_Report_CSVs/2021_DE_Region_Mobility_Report.csv")
df2022 = pd.read_csv("Region_Mobility_Report_CSVs/2022_DE_Region_Mobility_Report.csv")
df = pd.concat([df2020,df2021,df2022])
#apple
df_apple= pd.read_csv("https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/apple_reports/apple_mobility_report.csv")
df_apple.date = pd.to_datetime(df_apple.date)
df.date = pd.to_datetime(df.date)


In [ ]:
set(df_apple.country)

In [ ]:
df_apple_ge = df_apple[df_apple['country']=="Germany"][df_apple['sub-region']=="Total"].set_index("date")


features_apple =  ['driving','transit','walking']
df_apple_ge=df_apple_ge[features_apple]
features = ['retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']
df_Ger = df[df["sub_region_1"].isnull()].set_index('date')[features]

df2 = pd.read_csv("cx_Germany.csv")
df2.day = pd.to_datetime(df2.day)



df = df_Ger.join(df2.set_index("day")).rolling(7).sum().multiply(1/7).shift(-3).dropna()

df_a = df_apple_ge.join(df2.set_index("day")).rolling(7).sum().multiply(1/7).shift(-3).dropna()

df = df.join(df_apple_ge).dropna()

#df = (df-df.shift(-1)).dropna()

import xgboost as xgb
dtrain = xgb.DMatrix(df[features+features_apple][:700], label=df['cx'][:700])
dtest_reg = xgb.DMatrix(df[features+features_apple])

# Define hyperparameters

params = {"objective": "reg:squarederror", "tree_method": "hist"} #,"booster":"gblinear" }
 #,"booster":"gblinear" 

n = 20
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

In [ ]:
# Government response index
df_gri= pd.read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_nat_latest.csv")

In [ ]:
df_gri

In [ ]:
df_gri.Date = pd.to_datetime(df_gri.Date,format="%Y%m%d")
df_gri = df_gri.set_index('Date')
#df_gri[df_gri['CountryName']=='Germany'].StringencyIndex_Average.plot()


In [ ]:
def predict_ctr(ctr_sh,ctr_ln,R=True,GRI=False,bis="2021-01-31"):
    df2020_c = pd.read_csv("Region_Mobility_Report_CSVs/2020_"+ctr_sh+"_Region_Mobility_Report.csv")
    df2021_c = pd.read_csv("Region_Mobility_Report_CSVs/2021_"+ctr_sh+"_Region_Mobility_Report.csv")
    df2022_c = pd.read_csv("Region_Mobility_Report_CSVs/2022_"+ctr_sh+"_Region_Mobility_Report.csv")

    df_c = pd.concat([df2020_c,df2021_c,df2022_c])
    df_c.date = pd.to_datetime(df_c.date)

    df_apple_c = df_apple[df_apple['country']==ctr_ln][df_apple['sub-region']=="Total"].set_index("date")
    df_apple_c=df_apple_c[features_apple]

    df_c = df_c[df_c["sub_region_1"].isnull()].set_index('date')[features]

    df_r = df_r_all[df_r_all.location==ctr_ln][['date','reproduction_rate']].set_index('date').dropna()

    df_c = df_c.join(df_apple_c).dropna()
    df_c = df_c.join(df_r).dropna()
    df_c = df_c.join(df_gri[df_gri['CountryName']==ctr_ln].StringencyIndex_Average).dropna()

    dtest_reg = xgb.DMatrix(df_c[features+features_apple])

    preds = model.predict(dtest_reg)

    df_c['preds'] = preds
    plt.figure()
    df_c.preds.rolling(7).sum().multiply(1/7).shift(-3).plot(label="Contact Index",linewidth=2)
    top = max(df_c.preds.rolling(7).sum().multiply(1/7)["2020-04-15":bis].values)
    bot = min(df_c.preds.rolling(7).sum().multiply(1/7)["2020-04-15":bis].values)
    plt.ylim(0.8*bot,1.1*top)
    print (bot,top)
    plt.xlim("2020-02-15",bis)
    plt.ylabel("Contact Index")
    plt.title = ctr_ln
    ax=plt.gca()
    
    if R:
    #ax2.add_patch(Rectangle((50, 15), 1, 55,alpha=0.5))
    #ax2.add_patch(Rectangle((304, 15), 1, 55,alpha=0.5))
    #plt.scatter(["2020-11-05"]*50,np.arange(20,50,30./50),color="k",s=1)
    #plt.scatter(["2021-01-06"]*50,np.arange(20,50,30./50),color="k",s=1)

        ax2 = df_c.reproduction_rate.plot(secondary_y=True,label="effective R",linestyle = "--")
        top = max(df_c.reproduction_rate["2020-05-01":"2020-12-31"])
        bot = min(df_c.reproduction_rate["2020-05-01":"2020-12-31"])
        plt.ylim(0.9*bot,1.1*top)
        plt.ylabel("Effective R")
        lines = ax.get_lines() + ax.right_ax.get_lines()
        ax.legend(lines,["CX","effective R"],loc=1)
#        plt.legend(loc=1)

    if GRI:
        ax2 = df_c.StringencyIndex_Average.dropna().plot(secondary_y=True,label="GRI",linestyle = "--")
        top = max(df_c.StringencyIndex_Average["2020-05-01":bis].dropna())#["2020-05-01":"2020-12-31"])
        bot = min(df_c.StringencyIndex_Average["2020-05-01":bis].dropna()) #["2020-05-01":"2020-12-31"])
        plt.ylim(1.02*top,0.8*bot)
        plt.ylabel("GRI")
        lines = ax.get_lines() + ax.right_ax.get_lines()
        ax.legend(lines,["CX","GRI"],loc=2)

        
        
    ax.set_title(ctr_ln)


In [ ]:
ctrs = { "FR": "France", "GB": "United Kingdom","US": "United States",
       "IT": "Italy", "ES": "Spain"}

#fig, axs = plt.subplots(2)
#i=0
for ctr_sh in ctrs:    
    ctr_ln = ctrs[ctr_sh]
    predict_ctr(ctr_sh,ctr_ln,False,True,"2022-04-30")
